In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
import skvideo
skvideo.setFFmpegPath("C://FFmpeg/bin/")
import skvideo.io  
import GPUtil

In [2]:
def loader(path):
    vid = skvideo.io.vread(path)  
    videodata= np.transpose(vid, (3, 0, 1, 2))
    #print(videodata.shape)
    del vid
    return videodata

In [3]:
data_dir = 'MS-ASL100'

video_datasets = {x: datasets.DatasetFolder(os.path.join(data_dir, x), loader=loader, extensions='.mp4')
                  for x in ['train', 'val']}
# print(len(video_datasets['train']))
dataloaders = {x: torch.utils.data.DataLoader(video_datasets[x], batch_size=1, shuffle=True, num_workers=0)
              for x in ['train', 'val']}
 #print(len(dataloaders['train']))
dataset_sizes = {x: len(video_datasets[x]) for x in ['train', 'val']}
class_names = video_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        torch.cuda.empty_cache()
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                inputs = inputs.float()
#                 print(list(inputs.size()))
                labels = labels.to(device)
#                 print(list(inputs.size()))
                
                # zero the parameter gradients
                optimizer.zero_grad()
                model.zero_grad()
                
                # track history if only in train
                torch.cuda.empty_cache()
                with torch.set_grad_enabled(phase == 'train'):
#                     GPUtil.showUtilization(all=True)
                    outputs = model(inputs)
#                     print(outputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.detach().item()
                running_corrects += torch.sum(preds == labels.data).detach().item()
                
                del inputs
                del labels
                del loss
                del outputs
                del preds
                
                if(i%100==0):
                    print('Batch: {}'.format(i))
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), "aslModel.pth.tar")
    return model

In [8]:
model_ft = torchvision.models.video.r2plus1d_18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
model_ft = model_ft.to(device)

weightTensor = torch.from_numpy(np.loadtxt('weightsArray.txt'))
loss = nn.CrossEntropyLoss(weight = weightTensor)
loss = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0, weight_decay=0.01)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.01)

In [9]:
torch.cuda.empty_cache()
weightTensor.detach()
model_ft = train_model(model_ft, loss, optimizer_ft, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
[1, 3, 59, 100, 177]


RuntimeError: Could not run 'aten::slow_conv3d_forward' with arguments from the 'CUDATensorId' backend. 'aten::slow_conv3d_forward' is only available for these backends: [CPUTensorId, VariableTensorId].

In [ ]:
arr = np.zeros(len(class_names))
counter = 0
for _, className in video_datasets['train']:
    if (counter%100 == 0):
        print(counter)
    arr[className] += 1
    counter += 1
sum = np.sum(arr)
arr = np.subtract(sum, arr)
weightsArray = np.true_divide(arr, sum)
# print(weightsArray)

In [12]:
def counter_parameters(model):
    return sum(p.numel() for p in model.parameters())
count = counter_parameters(model_ft)
print(count)

31556625
